In [1]:
import requests
from time import sleep
import pandas as pd
import numpy as np

from keras.layers import Input, Embedding, Flatten, Dot
from keras.models import Model
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [ ]:
#env = 'http://35.180.46.68/''
env = 'http://35.180.254.42/'

In [2]:
USER_ID = '9G08LOYFU88BJ8GHNRU3'

In [3]:
#http://35.180.16.68/predict?user_id=aaaa&predicted_score=0.761
#http://35.180.16.68/reset?user_id=aaaa

In [4]:
r = requests.get(url=env+'reset', params= {'user_id':USER_ID})
sleep(0.05)
nb_items = r.json()['nb_items']
nb_users = r.json()['nb_users']
next_item = r.json()['next_item']
next_user = r.json()['next_user']
item_history = r.json()['item_history']
user_history = r.json()['user_history']
rating_history = r.json()['rating_history']

In [5]:
ratings = pd.DataFrame(data={'user_id':user_history, 'item_id':item_history, 'rating': rating_history})

In [6]:
ratings.head()

,user_id,item_id,rating
0,29,296,3
1,0,282,2
2,17,270,1
3,62,81,4
4,32,87,5


In [7]:
user_id_input = Input(shape=[1],name='user')
item_id_input = Input(shape=[1], name='item')

embedding_size = 5
user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                           input_length=1, name='user_embedding')(user_id_input)

item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                           input_length=1, name='item_embedding')(item_id_input)

user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)

y = Dot(axes=1)([user_vecs, item_vecs])

model = Model(inputs=[user_id_input, item_id_input], outputs=y)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 5)         505         user[0][0]                       
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 5)         1505        item[0][0]                       
__________________________________________________________________________________________________
flatten_1 

In [9]:
model.compile(optimizer='adam', loss='MSE')

In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit([ratings['user_id'], ratings['item_id']], ratings['rating'],
                    batch_size=64, epochs=20, validation_split=0.1,
                    shuffle=True, callbacks=[early_stopping])

Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 0s 28us/step - loss: 9.6570 - val_loss: 9.9388
Epoch 2/20
9000/9000 [==============================] - 0s 14us/step - loss: 9.6160 - val_loss: 9.8043
Epoch 3/20
9000/9000 [==============================] - 0s 13us/step - loss: 9.2005 - val_loss: 9.0123
Epoch 4/20
9000/9000 [==============================] - 0s 14us/step - loss: 8.0223 - val_loss: 7.4776
Epoch 5/20
9000/9000 [==============================] - 0s 14us/step - loss: 6.3231 - val_loss: 5.6894
Epoch 6/20
9000/9000 [==============================] - 0s 14us/step - loss: 4.6332 - val_loss: 4.0981
Epoch 7/20
9000/9000 [==============================] - 0s 14us/step - loss: 3.2487 - val_loss: 2.8645
Epoch 8/20
9000/9000 [==============================] - 0s 14us/step - loss: 2.2407 - val_loss: 2.0039
Epoch 9/20
9000/9000 [==============================] - 0s 14us/step - loss: 1.5628 - val_loss: 1.4360
Epoch 10/20
9000/9000 [==

In [11]:
nb_samples = 1000
mse = 0
users_list = []
ratings_list = []
items_list = []
for i in range(nb_samples):
    sleep(0.05)
    predicted_score = model.predict([[next_user], [next_item]])[0,0]
    r = requests.get(url=env + 'predict', params= {'user_id':USER_ID, 'predicted_score':predicted_score})
    true_rating = r.json()['rating']
    mse += (true_rating - predicted_score)**2
    users_list += [next_user]
    ratings_list += [true_rating]
    items_list += [next_item]
    if (i+1)%100 == 0:
        model.fit([users_list,items_list], ratings_list, epochs=20)
        users_list = []
        ratings_list = []
        items_list = []
    next_item = r.json()['next_item']
    next_user = r.json()['next_user']
print('mse: ', mse/nb_samples )

Train on 90 samples, validate on 10 samples
Epoch 1/20
90/90 [==============================] - 0s 57us/step - loss: 0.5115 - val_loss: 0.2508
Epoch 2/20
90/90 [==============================] - 0s 95us/step - loss: 0.5013 - val_loss: 0.2497
Epoch 3/20
90/90 [==============================] - 0s 87us/step - loss: 0.4852 - val_loss: 0.2486
Epoch 4/20
90/90 [==============================] - 0s 77us/step - loss: 0.4652 - val_loss: 0.2480
Epoch 5/20
90/90 [==============================] - 0s 76us/step - loss: 0.4452 - val_loss: 0.2473
Epoch 6/20
90/90 [==============================] - 0s 122us/step - loss: 0.4200 - val_loss: 0.2466
Epoch 7/20
90/90 [==============================] - 0s 82us/step - loss: 0.3988 - val_loss: 0.2460
Epoch 8/20
90/90 [==============================] - 0s 80us/step - loss: 0.3768 - val_loss: 0.2454
Epoch 9/20
90/90 [==============================] - 0s 137us/step - loss: 0.3572 - val_loss: 0.2449
Epoch 10/20
90/90 [==============================] - 0s 106us/s